In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\suraj\\medical_chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\suraj\\medical_chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader

    )
    documents=loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="DATA/")

In [8]:
# split the data into text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of text chunks", len(text_chunks))

length of text chunks 7023


In [10]:
# embedding model from huggingFace
from langchain.embeddings import HuggingFaceBgeEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\suraj\AppData\Local\Temp\ipykernel_24140\2499133308.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Suraj\Anaconda3\envs\medical_chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("hello_world")
print("length", len(query_result))

length 384


In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [15]:
# only run for first time running again will give error like the one below
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "medicalchatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


{
    "name": "medicalchatbot",
    "metric": "cosine",
    "host": "medicalchatbot-uc347b4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [16]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [17]:
# embed each chunk abd upsert the embeddings into your Pinecone index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,

)

In [18]:
# load the existing index
from langchain_pinecone import PineconeSparseVectorStore

# embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
 

In [19]:
docsearch

In [20]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs={"k":3})

In [21]:
retrieved_docs = retriever.invoke("what is acne?")

In [22]:
retrieved_docs

[Document(id='f72b4d9f-eb19-4f03-b1b4-bd66b44608a6', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 423.0, 'page_label': '424', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'DATA\\Medical_book.pdf', 'total_pages': 759.0}, page_content='thing that irritates the skin and is manifested by one or\nmore lines of red, swollen, blistered skin that may itch or\nGALE ENCYCLOPEDIA OF MEDICINE 21036\nDermatitis'),
 Document(id='4ccfdd10-be7a-4b08-bfa3-15c206ef2f7a', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 297.0, 'page_label': '298', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'DATA\\Medical_book.pdf', 'total_pages': 759.0}, page_content='repeated exposure to an allergen (an allergy-causing sub-\nstance) triggers an immune response that inflames the\nskin. Tens of thousands of drugs, pesticides, cosmetics,\nfood additives, commerc

In [23]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=500)

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "You are an assistant for question-answering tasks."
    "use the following pieces of retrieved context to answer"
    "the question.If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"

)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ('human',"{input}")
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [26]:
# jab pasia aur credits ho tab isko run karke dekh lena 

response = rag_chain.invoke({'input':"what is acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}